In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv("../input/sf-crime/train.csv.zip")
test_data = pd.read_csv("../input/sf-crime/test.csv.zip")

train_data.shape
test_data.shape
test_ids = test_data['Id']
train_data.head(10)
test_data.head(10)

train_data.info()
train_data.isnull().sum()



#Removing Unwanted Locations
train_data.drop(train_data[train_data['Y']>40].index,inplace = True)


#Dates Cleaning
#Dates is string type

def DataCleaning(df):
    #Duplicates
    df.duplicated().sum()
    df.drop_duplicates(inplace = True)
    df['Dates'] = pd.to_datetime(df['Dates'])
    df["Year"] = df.Dates.dt.year
    df['Month'] = df.Dates.dt.month
    df['Hour'] = df.Dates.dt.hour
    
    df['DayOfWeekint'] =df['DayOfWeek'].map({
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7      })    
    
       
    return df
    
clean_train = DataCleaning(train_data)
clean_test = DataCleaning(test_data)    


gb = train_data.groupby("Year")['Descript'].count()
gb.plot(xticks = gb.index.values)
plt.ylabel("Count of crimes")

for y in train_data['Year'].unique():
    train_data[train_data['Year']==y].groupby('DayOfWeek')['Descript'].count().sort_values().plot(kind = 'barh',title = 'Count Of Crimes Per Day')
    plt.show()
for y in train_data['Year'].unique():
    train_data[train_data['Year']==y].groupby('Month')['Descript'].count().sort_values().plot(kind = 'barh',title = 'Count Of Crimes in {}'.format(y))
    plt.show()

train_data['IsDay'] = train_data.Hour.apply(lambda x : 1 if x>7 & x<20 else 0)

test_data['IsDay'] = test_data.Hour.apply(lambda x : 1 if x>7 & x<20 else 0)

for y in train_data['Year'].unique():
    train_data[train_data['Year']==y].groupby('IsDay')['Descript'].count().sort_values().plot(kind = 'bar',title = 'Count Of Crimes D/N {}'.format(y))
    plt.show()

plt.figure(figsize = (20,20))
train_data['Category'].value_counts().sort_values().plot(kind = 'barh')
plt.xlabel("Count")
plt.ylabel("Categories")
plt.show()

plt.figure(figsize = (20,20))
train_data['PdDistrict'].value_counts().sort_values().plot(kind = 'barh')
plt.xlabel("Count")
plt.ylabel("PdDistrict")
plt.show()

ct = pd.crosstab(train_data.Category, train_data.IsDay)
ct.plot(kind='barh', figsize=(14,14), title='Crime Categories by Day/Night (2003 ~ mid 2015)')

In [ ]:
clean_train.drop(['Resolution','Descript'],axis = 'columns',inplace = True) 
clean_train.drop(['DayOfWeek'],axis = 'columns',inplace = True) 
clean_test.drop(['DayOfWeek','Id'],axis = 'columns',inplace = True) 


clean_train['Block'] = clean_train['Address'].str.contains('block', case=False)
clean_train['IsBlock'] = clean_train.Block.apply(lambda x : 1 if x ==True else 0)
clean_train.drop(['Address','Block'],axis = 'columns',inplace = True) 

clean_test['Block'] = clean_test['Address'].str.contains('block', case=False)
clean_test['IsBlock'] = clean_test.Block.apply(lambda x : 1 if x ==True else 0)
clean_test.drop(['Address','Block'],axis = 'columns',inplace = True) 

#PreProcessing
categorical_features = [feature for feature in clean_train.columns if clean_train[feature].dtypes == 'O' and feature not in ['Category']]
len(categorical_features)
for features in categorical_features:
    dummies = pd.get_dummies(clean_train[features])
    merged = pd.concat([clean_train,dummies],axis = 'columns')
    clean_train_pro = merged.copy()



categorical_features_test = [feature for feature in clean_test.columns if clean_test[feature].dtypes == 'O']
     
for features in categorical_features_test:
    dummies = pd.get_dummies(clean_test[features])
    merged = pd.concat([clean_test,dummies],axis = 'columns')
    clean_test_pro = merged.copy()

from sklearn.preprocessing import LabelEncoder
cat_le = LabelEncoder()
clean_train_pro['CategoryInt'] = (cat_le.fit_transform(clean_train_pro.Category))

clean_train_pro.drop(['PdDistrict','Category','Dates'],axis = 'columns',inplace = True) 
clean_test_pro.drop(['PdDistrict','Dates'],axis = 'columns',inplace = True) 

X_train = clean_train_pro.drop('CategoryInt',axis = 'columns')
y_train = clean_train_pro['CategoryInt']

X_test = clean_test_pro



In [ ]:
import xgboost as xgb
train_xgb = xgb.DMatrix(X_train, label=y_train)
test_xgb  = xgb.DMatrix(X_test)

In [ ]:

params = {
    'max_depth': 4,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 39,
}

In [ ]:
CROSS_VAL = False
if CROSS_VAL:
    print('Doing Cross-validation ...')
    cv = xgb.cv(params, train_xgb, nfold=3, early_stopping_rounds=10, metrics='mlogloss', verbose_eval=True)
    cv


In [ ]:
test_data.info()

In [ ]:

SUBMIT = not CROSS_VAL
if SUBMIT:
    print('Fitting Model ...')
    m = xgb.train(params, train_xgb, 10)
    res = m.predict(test_xgb)
    cols = ['Id'] + cat_le.classes_
    submission = pd.DataFrame(res, columns=cat_le.classes_)
    submission.insert(0, 'Id', test_ids)
    submission.to_csv('submission.csv', index=False)
    print('Done Outputing !')
    print(submission.sample(3))
else:
    print('NOT SUBMITING')